<a href="https://colab.research.google.com/github/04pys/cs-systems-labs/blob/main/notebooks/phase01_lowlevel_basics/lab01_cache_walk/phase01_lab01_cache_walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
cat > main.cpp << 'CPP'
CPP

In [ ]:
!ls -l

total 4
-rw-r--r-- 1 root root    0 Jan  1 07:22 main.cpp
drwxr-xr-x 1 root root 4096 Dec 11 14:34 sample_data


In [72]:
%%bash
cat > main.cpp << 'CPP'
#include <bits/stdc++.h>
using namespace std;

static inline long long now_ns() {
  return chrono::duration_cast<chrono::nanoseconds>(
    chrono::steady_clock::now().time_since_epoch()
  ).count();
}

int main() {
  ios::sync_with_stdio(false);
  cin.tie(nullptr);

  int N = 8192; // 필요하면 2048, 4096, 8192로 바꿔가며 실험 가능함
  vector<int> a((long long)N * N, 1);

  auto bench = [&](const string& name, bool row_major) {
    volatile long long sum = 0; // 최적화로 루프가 사라지는 것 방지
    long long t0 = now_ns();

    if (row_major) {
      for (int i = 0; i < N; i++)
        for (int j = 0; j < N; j++)
          sum += a[(long long)i * N + j];
    } else {
      for (int j = 0; j < N; j++)
        for (int i = 0; i < N; i++)
          sum += a[(long long)i * N + j];
    }

    long long t1 = now_ns();
    double ms = (t1 - t0) / 1e6;
    double ns = (t1 - t0);
    cout << name << " N=" << N << " time_ms=" << ms << " sum=" << sum << "\n";
  };

  // 워밍업(첫 실행은 캐시/페이지 폴트 등으로 흔들릴 수 있음)
  bench("warmup_row", true);

  // 측정 2개
  bench("row_major", true);
  bench("col_major_like", false);

  return 0;
}
CPP

In [73]:
!g++ -O2 -std=c++17 main.cpp -o main

In [ ]:
!./main

warmup_row N=4096 time_ms=7.79522 sum=16777216
row_major N=4096 time_ms=7.92456 sum=16777216
col_major_like N=4096 time_ms=102.229 sum=16777216


In [ ]:
!./main

warmup_row N=4 time_ns=200 sum=16
row_major N=4 time_ns=120 sum=16
col_major_like N=4 time_ns=120 sum=16


In [62]:
!./main

warmup_row N=256 time_ms=0.04635 sum=65536
row_major N=256 time_ms=0.03391 sum=65536
col_major_like N=256 time_ms=0.08958 sum=65536


In [65]:
!./main

warmup_row N=512 time_ms=0.17416 sum=262144
row_major N=512 time_ms=0.11723 sum=262144
col_major_like N=512 time_ms=0.44543 sum=262144


In [68]:
!./main

warmup_row N=1024 time_ms=0.43342 sum=1048576
row_major N=1024 time_ms=0.45221 sum=1048576
col_major_like N=1024 time_ms=3.98134 sum=1048576


In [71]:
!./main

warmup_row N=2048 time_ms=1.89672 sum=4194304
row_major N=2048 time_ms=1.90973 sum=4194304
col_major_like N=2048 time_ms=24.7508 sum=4194304


In [74]:
!./main

warmup_row N=8192 time_ms=33.6993 sum=67108864
row_major N=8192 time_ms=32.7658 sum=67108864
col_major_like N=8192 time_ms=1304.5 sum=67108864
